In [1]:
from __future__ import print_function, division   # Python 2/3 compatibility
from skimage import io                            # utilities to read and write images in various formats
import numpy as np  # array manipulation package
import matplotlib.pylab as plt                    # plotting package
%matplotlib inline
plt.rcParams['figure.figsize'] = (8,8)         # set default figure size
plt.rcParams['image.cmap'] = 'gray'               # set default colormap to gray

# Assignment 3 : Frequency domain processing

The following progamming assignment involves the task of image filtering in the frequency domain, such as, highpass, lowpass and bandpass filtering.

**Please, follow carefully the submission instructions given in the end of this notebook.** You are encouraged to seek information in other places than the course book and lecture material but remember **list all your sources under references**.

If you experience problems that you cannot solve using the course material or the Python documentation, or have any questions regarding the programming assignments, please do not hesitate to contact the course assistant by sending an e-mail at dip@unioulu.oulu.fi. You can also join in for the Q & A session (schedule is given on the course page in Moodle) for this assignment.

**Please, fill in your personal details below.**

# Personal details:

* **Name(s) and student ID(s):** 
    Haseeb Ur Rehman (2315255) and Marwa Bibi (2407704)
* **Contact information:** 
    Haseeb.Rehman@student.oulu.fi and Marwa.Bibi@student.oulu.fi

# 1. Image transforms : lowpass and highpass filtering in frequency domain

In the following, you will first perform ideal lowpass and highpass filtering on the test image, and later, we will also consider the Gaussian lowpass and highpass filtering. First, read the part concerning image enhancement in frequency domain in the lecture notes or in the course book. Specifically, you should look at the **Chapter-4** (available as a PDF file) in the lecture notes in Moodle.

Now, perform the following operations in the reserved code cells and answer to the questions written in bold into the reserved spaces.


**1.1. Read and display the test image `hplptest.jpg`.**

In [ ]:
# read test image
# read test image
image = io.imread('hplptest.jpg')

# display test image
plt.imshow(image, cmap='gray')
plt.title('Original Image')
plt.axis('off')
plt.show()


**1.2. Compute the Fourier transform (FT) of the test image and take a look at what the magnitude of the FT looks like.**

Hint: When plotting the FTs, use logarithmic graylevel transformation to make the result more illustrative for human visual system: 

`>>> np.log(np.abs(image_fft)+1)`

In [ ]:
from scipy import fftpack

# compute the FT of the test image using 'fftpack.fft2'

image_fft = fftpack.fft2(image)

# translate the origin of the FT (low frequencies) to the center using 'ftpack.fftshift'
image_fft_shifted = fftpack.fftshift(image_fft)

# display the magnitude of the uncentered and centered FT using 'imshow'.

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.imshow(np.log(np.abs(image_fft) + 1), cmap='gray')
plt.title('Magnitude of Uncentered FT')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(np.log(np.abs(image_fft_shifted) + 1), cmap='gray')
plt.title('Magnitude of Centered FT')
plt.axis('off')

plt.show()


**The code for constructing an ideal lowpass filter is given below:**

In [4]:
# make two frequency matrices, 'f1' and 'f2', as help variables (frequencies from -1 to 1)
n = (500,500)
f1 = ( np.arange(0,n[0])-np.floor(n[0]/2) ) * (2./(n[0]))
f2 = ( np.arange(0,n[1])-np.floor(n[1]/2) ) * (2./(n[1]))
f1, f2 = np.meshgrid(f1, f2)

# make a matrix with absolute values of frequency (“sampled” frequency domain)
D = np.sqrt(f1**2 + f2**2)

# set cut-off frequency D0 to 0.2
D0 = 0.2;

# filter matrix is initialized to ones 
Hlp = np.ones(n)

# set frequencies in filter mask Hlp greater than the cut-off frequency D0 to zero, other elements remain unaltered
Hlp[D>D0] = 0.0

**1.3. Modify the lowpass filter code and construct ideal highpass filter `Hhp` with the same cut-off frequency `D0=0.2` and display both ideal lowpass and highpass filter masks in the same figure.**

In [ ]:
# create ideal highpass filter mask Hhp

Hhp = np.zeros(n)
Hhp[D > D0] = 1.0

# display the filters

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.imshow(Hlp, cmap='gray')
plt.title('Ideal Lowpass Filter Mask')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(Hhp, cmap='gray')
plt.title('Ideal Highpass Filter Mask')
plt.axis('off')

plt.show()

**1.4. Perform ideal lowpass and highpass filtering in the frequency domain by multiplying the centralized FT of the original image with the `Hlp` and `Hhp` filter masks (element-per-element matrix multiplication) and display the two resulting FTs in the same figure.**

In [ ]:
# apply ideal lowpass and highpass filtering to the test image, i.e. multiply element-wise the fft of the image with the filter masks

filtered_lowpass = image_fft_shifted * Hlp
filtered_highpass = image_fft_shifted * Hhp


# display the magnitude of the resulting FTs


plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.imshow(np.log(np.abs(filtered_lowpass) + 1), cmap='gray')
plt.title('Magnitude of Lowpass Filtered FT')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(np.log(np.abs(filtered_highpass) + 1), cmap='gray')
plt.title('Magnitude of Highpass Filtered FT')
plt.axis('off')

plt.show()

**1.5. Reconstruct the filtered images with `fftpack.ifft2()` and `fftpack.ifftshift()` in reverse order and display the two filtered images using `imshow()` in the same figure.** 

Hint: Due to round-off errors, you have to take the real part of the result of inverse FT before displaying it with `imshow()`. Please note also that the resulting images values beyond the original `uint8` image `[0,255]`, so you need to clip these values using `np.clip()`.

In [ ]:
# reconstruct the filtered images
filtered_lowpass_shifted_back = fftpack.ifftshift(filtered_lowpass)
filtered_highpass_shifted_back = fftpack.ifftshift(filtered_highpass)


# take the 'real' part of the resulting images due to possible round-off errors

filtered_lowpass_image = np.real(fftpack.ifft2(filtered_lowpass_shifted_back))
filtered_highpass_image = np.real(fftpack.ifft2(filtered_highpass_shifted_back))


# clip values beyond the uint8 range [0,255] 


filtered_lowpass_image = np.clip(filtered_lowpass_image, 0, 255).astype(np.uint8)
filtered_highpass_image = np.clip(filtered_highpass_image, 0, 255).astype(np.uint8)

# display the original image and its lowpass and highpass filtered images in the same figure

plt.figure(figsize=(12, 6))

plt.subplot(1, 3, 1)
plt.imshow(image, cmap='gray')
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(filtered_lowpass_image, cmap='gray')
plt.title('Lowpass Filtered Image')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(filtered_highpass_image, cmap='gray')
plt.title('Highpass Filtered Image')
plt.axis('off')

plt.show()

When performing ideal lowpass and highpass filtering, unwanted artefacts appear to the filtered image. **What is this phenomenon called and why does it occur?**

`The unwanted artefacts are called ringing effects . They occur because the ideal lowpass and highpass filters have sharp transitions in the frequency domain, which introduce oscillations in the spatial domain due to the Gibbs phenomenon.`

**1.6. Now, construct Gaussian lowpass and highpass filters with cut-off frequency `D0=0.2` and display them in the same figure.**

Hint: All you need to do is to modify the filter matrix `Hlp` line in the example code snippet accordingly to form `Hlpg` and `Hhpg` (see the lecture notes, **chapter04.pdf**).

In [ ]:
# Construct a Gaussian lowpas and a highpass filter
Hlpg = np.exp(-(D**2) / (2 * (D0**2)))

Hhpg = 1 - Hlpg
# display the filter masks

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.imshow(Hlpg, cmap='gray')
plt.title('Gaussian Lowpass Filter Mask')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(Hhpg, cmap='gray')
plt.title('Gaussian Highpass Filter Mask')
plt.axis('off')

plt.show()

**1.7. Perform Gaussian lowpass and highpass filtering to the original test image and display the magnitude of the resulting FTs in the same figure.**

In [ ]:
# apply gaussian lowpass and highpass filtering to the test image

filtered_gaussian_lowpass = image_fft_shifted * Hlpg
filtered_gaussian_highpass = image_fft_shifted * Hhpg

# display the magnitude of the resulting FTs


plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.imshow(np.log(np.abs(filtered_gaussian_lowpass) + 1), cmap='gray')
plt.title('Magnitude of Gaussian Lowpass Filtered FT')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(np.log(np.abs(filtered_gaussian_highpass) + 1), cmap='gray')
plt.title('Magnitude of Gaussian Highpass Filtered FT')
plt.axis('off')

plt.show()

**1.8. Finally, reconstruct the filtered images like in step 4.5. and display the original image and the two Gaussian filtered images in the same figure.**

In [ ]:
# reconstruct the filtered images 

filtered_gaussian_lowpass_shifted_back = fftpack.ifftshift(filtered_gaussian_lowpass)
filtered_gaussian_highpass_shifted_back = fftpack.ifftshift(filtered_gaussian_highpass)

filtered_gaussian_lowpass_image = np.real(fftpack.ifft2(filtered_gaussian_lowpass_shifted_back))
filtered_gaussian_highpass_image = np.real(fftpack.ifft2(filtered_gaussian_highpass_shifted_back))

filtered_gaussian_lowpass_image = np.clip(filtered_gaussian_lowpass_image, 0, 255).astype(np.uint8)
filtered_gaussian_highpass_image = np.clip(filtered_gaussian_highpass_image, 0, 255).astype(np.uint8)

# display the three images in the same figure

plt.figure(figsize=(12, 6))

plt.subplot(1, 3, 1)
plt.imshow(image, cmap='gray')
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(filtered_gaussian_lowpass_image, cmap='gray')
plt.title('Gaussian Lowpass Filtered Image')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(filtered_gaussian_highpass_image, cmap='gray')
plt.title('Gaussian Highpass Filtered Image')
plt.axis('off')

plt.show()

**Do the unwanted artefacts appear in the Gaussian lowpass filtered image, why or why not?**

`No, the ringing effect does not appear in the Gaussian lowpass filtered image because the Gaussian filter has a smooth transition in the frequency domain, avoiding sharp edges that cause the Gibbs phenomenon.`

**What kind of effect does Gaussian (and ideal) lowpass filtering have on images in general? Why? What about highpass filtering? Why?**

`Effect of Lowpass and Highpass Filtering`

`Lowpass filtering : Smooths the image by attenuating high-frequency components, reducing noise and fine details. This results in a blurred image.`

`Highpass filtering : Enhances edges and fine details by attenuating low-frequency components, making the image sharper.`

# Aftermath
Finally, fill your answers to the following questions:

**How much time did you need to complete this exercise?**

`We spend more than 5+ hours.`

**Did you experience any problems with the exercise? Was there enough help available? Should this notebook be more (or less) detailed?**

`None significant, the instructions were clear`

# References
`Lecture notes: Chapter 4 of Book Digitsl Image Processing`

`Python documentation for numpy, scipy.fftpack, and matplotlib`

# Submission

1. Before submitting your work, **check that your notebook (code) runs from scratch** and reproduces all the requested results by clicking on the menu `Kernel -> Restart & Run All`! Also, check that you have answered all the questions written in **bold**.
2. Clear all outputs and variables, etc. by click on the menu `Kernel -> Restart & Clear Output`. This may (or will) reduce the file size of your deliverable a lot! 
3. Rename this Jupyter notebook to **`DIP_PA3_[student number(s)].ipynb`** (e.g. `DIP_PA3_1234567.ipynb` if solo work or `DIP_PA3_1234567-7654321.ipynb` if pair work) and upload it as your submission to Moodle.